In [2]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os



In [3]:
data_path = os.path.join("..", "data")

In [4]:
X = np.load(data_path + '/initial_data/function_6/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_6/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.7281861  0.15469257 0.73255167 0.69399651 0.05640131]
 [0.24238435 0.84409997 0.5778091  0.67902128 0.50195289]
 [0.72952261 0.7481062  0.67977464 0.35655228 0.67105368]
 [0.77062024 0.11440374 0.04677993 0.64832428 0.27354905]
 [0.6188123  0.33180214 0.18728787 0.75623847 0.3288348 ]
 [0.78495809 0.91068235 0.7081201  0.95922543 0.0049115 ]
 [0.14511079 0.8966846  0.89632223 0.72627154 0.23627199]
 [0.94506907 0.28845905 0.97880576 0.96165559 0.59801594]
 [0.12572016 0.86272469 0.02854433 0.24660527 0.75120624]
 [0.75759436 0.35583141 0.0165229  0.4342072  0.11243304]
 [0.5367969  0.30878091 0.41187929 0.38822518 0.5225283 ]
 [0.95773967 0.23566857 0.09914585 0.15680593 0.07131737]
 [0.6293079  0.80348368 0.81140844 0.04561319 0.11062446]
 [0.02173531 0.42808424 0.83593944 0.48948866 0.51108173]
 [0.43934426 0.69892383 0.42682022 0.10947609 0.87788847]
 [0.25890557 0.79367771 0.6421139  0.19667346 0.59310318]
 [0.43216593 0.71561781 0.3418191  0.70499988 0.61496184]
 [0.78287

In [5]:
# add the week 1 input and  result to the array
X = np.append(X, [[0.736842,0.157895,0.736842,0.684211,0.052632]], axis=0)
y = np.append(y, [-0.74650183], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.368421, 0.263158, 0.526316, 0.842105, 0.105263]], axis=0)
y = np.append(y, [-0.32481759704902613], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.359946, 0.009339, 0.744659, 0.973816, 0.016177]], axis=0)
y = np.append(y, [-0.8381637411364704], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.412263, 0.328525, 0.520517, 0.793617, 0.125967]], axis=0)
y = np.append(y, [-0.12430862230397105], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.403200, 0.358308, 0.537100, 0.806405, 0.125919]], axis=0)
y = np.append(y, [-0.2224297640161302], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.391891, 0.368600, 0.531377, 0.802436, 0.120967]], axis=0)
y = np.append(y, [-0.2072371147670746], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.7281861  0.15469257 0.73255167 0.69399651 0.05640131]
 [0.24238435 0.84409997 0.5778091  0.67902128 0.50195289]
 [0.72952261 0.7481062  0.67977464 0.35655228 0.67105368]
 [0.77062024 0.11440374 0.04677993 0.64832428 0.27354905]
 [0.6188123  0.33180214 0.18728787 0.75623847 0.3288348 ]
 [0.78495809 0.91068235 0.7081201  0.95922543 0.0049115 ]
 [0.14511079 0.8966846  0.89632223 0.72627154 0.23627199]
 [0.94506907 0.28845905 0.97880576 0.96165559 0.59801594]
 [0.12572016 0.86272469 0.02854433 0.24660527 0.75120624]
 [0.75759436 0.35583141 0.0165229  0.4342072  0.11243304]
 [0.5367969  0.30878091 0.41187929 0.38822518 0.5225283 ]
 [0.95773967 0.23566857 0.09914585 0.15680593 0.07131737]
 [0.6293079  0.80348368 0.81140844 0.04561319 0.11062446]
 [0.02173531 0.42808424 0.83593944 0.48948866 0.51108173]
 [0.43934426 0.69892383 0.42682022 0.10947609 0.87788847]
 [0.25890557 0.79367771 0.6421139  0.19667346 0.59310318]
 [0.43216593 0.71561781 0.3418191  0.70499988 0.61496184]
 [0.78287

In [6]:
print(f"max y is :{np.max(y)}")

max y is :-0.12430862230397105


In [7]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

GaussianProcessRegressor(kernel=Matern(length_scale=1, nu=2.5),
                         normalize_y=True)

In [8]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
def ucb(x, gp, kappa=4.0):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
"""grid_size = 20
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
x3 = np.linspace(0, 1, grid_size)
x4 = np.linspace(0, 1, grid_size)
x5 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b, c, d, e] for a in x1 for b in x2 for c in x3 for d in x4 for e in x5])"""

In [9]:
n_samples = 3000000 # You can adjust this based on your compute budget

# Generate random 6D points in [0, 1]^6
X_random = np.random.rand(n_samples, 5)

In [10]:
ucb_values = np.array([ucb(x, gp) for x in X_random])


In [11]:
best_idx = np.argmax(ucb_values)
next_best_point = X_random[best_idx]
print("Next best point to evaluate (UCB) rounded:", np.round(next_best_point,6))
print("Next best point to evaluate (UCB):", next_best_point)
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Predicted maximum value (mean):", pred_mean[0])

Next best point to evaluate (UCB) rounded: [0.149263 0.157346 0.376126 0.401604 0.000412]
Next best point to evaluate (UCB): [0.14926303 0.15734618 0.37612566 0.40160358 0.00041168]
Predicted maximum value (mean): -0.5051072288786954


In [ ]:
# SVM model as surrogate
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=0.5, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 5)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)